## Feature Engineering Notebook 

### Import Libraries


In [1]:
import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from scipy.stats import normaltest

import math
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model  import LogisticRegression

from sklearn.neighbors import KNeighborsRegressor

# imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import (
    RandomUnderSampler,
    CondensedNearestNeighbour,
    TomekLinks,
    OneSidedSelection,
    EditedNearestNeighbours,
    RepeatedEditedNearestNeighbours,
    AllKNN,
    NeighbourhoodCleaningRule,
    NearMiss,
    InstanceHardnessThreshold
)

from imblearn.over_sampling import (
    RandomOverSampler,
    SMOTE,
    ADASYN,
    BorderlineSMOTE,
    SVMSMOTE,
)

from imblearn.combine import SMOTEENN, SMOTETomek


from imblearn.ensemble import (
    BalancedBaggingClassifier,
    BalancedRandomForestClassifier,
    RUSBoostClassifier,
    EasyEnsembleClassifier,
)

from sklearn.ensemble import (
    RandomForestClassifier,
    BaggingClassifier,
    AdaBoostClassifier,
)


# adding common folder location to sys.path
import sys
sys.path.append('../common')

from helper import get_config

### Loading Config

In [2]:
#config = get_config()

### Load Dataset

In [4]:
# path to your dataset, can be a csv file or xlsx
dataset_path = "../dataset/Bank_Personal_Loan_Modelling.xlsx"

## use code as per the type of data source

## use below line to read data from csv file
## df = pd.read_csv(dataset_path)
df = pd.read_excel(dataset_path, sheet_name = 1, index_col=0)

In [5]:
df.head()

,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
ID,,,,,,,,,,,,,
1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [6]:
target = 'Personal Loan'
# df_x = df.drop(columns=[target])
# df_y = df[target]

Let's separate out, numerical, categorical and numerical_normal and numerical_non_normal attributes<br/>
This list will be used for outliers treatment and other transformation later.

### Separating Numerical and Categorical attributes along with normal and non normal numerical attributes

In [7]:
# value counts method can be used to see if an attribute contains categorical data or continous data
unique_val_in_cols = df.apply( lambda col : col.nunique()).sort_values()
print(unique_val_in_cols)

Personal Loan           2
Securities Account      2
CD Account              2
Online                  2
CreditCard              2
Education               3
Family                  4
Age                    45
Experience             47
CCAvg                 108
Income                162
Mortgage              347
ZIP Code              467
dtype: int64


In [8]:
# decide reasnable threshold value for separating categorical and numerical attributes based on above result
threshold = 10

In [1]:
# to-do: add columsn for categorical and numerical in the dataset df, and read it

In [9]:
categorical_attributes = list(unique_val_in_cols[unique_val_in_cols < threshold].keys())
numerical_attributes = list(unique_val_in_cols[unique_val_in_cols > threshold].keys())

In [10]:
def normal_test(df, significance = .01):
    """
    Function to perform ks test and test against normal distribution using  
    D’Agostino, R. B. (1971), “An omnibus test of normality for moderate and large sample size”
    
    frame: a pandas dataframe
    significance: float. Alpha level for which the null hypotesis will be rejected (H0: series comes from a normal distribution)
    plot: Boolean, whether or not plot a histogram for resulting columns
    
    returns a dataframe with only those columns that follow a normal distribution according to test.
"""
    columns = df.columns.tolist()
    non_normal_columns = []

    for col in columns:
        aux = df[col]
    
        _, p = normaltest(aux)
    
        if p <= significance:
            # col is not normally distributed
            non_normal_columns.append(col)
        
    normal_columns = [cols for cols in columns if cols not in non_normal_columns]
    return normal_columns, non_normal_columns

In [11]:
normal_attributes, non_normal_attributes = normal_test(df[numerical_attributes])

In [12]:
print("Number of attributes:")
print("categorical:{0}".format(len(categorical_attributes)))
print("numerical:{0}".format(len(numerical_attributes)))
print("normal attributes:{0}".format(len(normal_attributes)))
print("non normal attributes:{0}".format(len(non_normal_attributes)))

Number of attributes:
categorical:7
numerical:6
normal attributes:0
non normal attributes:6


#### Removing Target column from the categorical list of variables so that it does not get transformed

In [13]:
categorical_attributes.remove(target)

### Perform basic data cleaning as per observations from EDA
* For this dataset, we know that the 'Experience' column has minor percentage of negative values(which is wrong) so let's treat it before using any other transformation methods.

In [14]:
print("Number of negative values: {0}".format(len(df[df['Experience'] < 0])))

df['Experience'] = df['Experience'].apply(lambda x : np.nan if x < 0 else x)

print("Number of negative values after imputing with NaN: {0}".format(len(df[df['Experience'] < 0])))
print("Number of NANs: {0}".format(df['Experience'].isna().sum()))

Number of negative values: 52
Number of negative values after imputing with NaN: 0
Number of NANs: 52


### 1. Outlier Treatment

#### 1.1 Outlier treatment for numerical attributes

Outlier treatment for Non-Normal distribution

In [15]:
#Treating outliers with zero coding-Any value less than zero will be made zero
def outliers_ZeroCoding(X,variable):
    X.loc[X[variable]<0, variable] = 0

In [16]:
#Treating outliers with top coding-Any value greater than maximum limit will be capped at maximum
def outliers_TopCoding_quantile(df,variable):
    # top coding: upper boundary for outliers according to interquantile proximity rule
    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
    Upper_fence = df[variable].quantile(0.75) + (IQR * 3)
    df.loc[df[variable]>Upper_fence, variable] = Upper_fence

In [17]:
#Treating outliers with top coding-Any value less than minimum limit will be capped at minimum
def outliers_BottomCoding_quantile(df,variable):
    # bottom coding: lower boundary for outliers according to interquantile proximity rule
    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
    Lower_fence = df[variable].quantile(0.25) - (IQR * 3)
    df.loc[df[variable]<Lower_fence, variable] = Lower_fence

In [18]:
for col in non_normal_attributes:
    outliers_TopCoding_quantile(df,col)
    outliers_BottomCoding_quantile(df,col)

Outlier treatment for Normal distribution

In [19]:
#Treating outliers with top coding-Any value greater than maximum limit will be capped at maximum
def outliers_TopCoding_gaussian(df,variable):
    # top coding: upper boundary for outliers according to gaussian rule
    Upper_fence = df[variable].mean()+3*df[variable].std()
    df.loc[df[variable]>Upper_fence, variable] = Upper_fence

In [20]:
#Treating outliers with top coding-Any value less than minimum limit will be capped at minimum
def outliers_BottomCoding_gaussian(df,variable):
    # bottom coding: lower boundary for outliers according to gaussian rule
    Lower_fence = df[variable].mean()-3*df[variable].std()
    df.loc[df[variable]<Lower_fence, variable] = Lower_fence

In [21]:
for col in normal_attributes:
    outliers_TopCoding_gaussian(df,col)
    outliers_BottomCoding_gaussian(df,col)

Convert the non-normal distribution to normal

In [22]:
def transform_BoxCox(df,variable):
    df[variable+'_boxcox'], param = stats.boxcox(df[variable])
    print('Optimal lambda: ', param)

#### 1.2 Outlier treatment for categorical attributes

In [23]:
def rare_new_imputation(df,variable,rare_cat):
    temp = df.groupby([variable])[variable].count()/np.float(len(df))
    rare_cat = [x for x in temp.loc[temp<0.05].index.values]
    df[variable+'_rare_imp'] = np.where(df[variable].isin(rare_cat), 'Others', df[variable])

In [24]:
def rare_freq_imputation(df,variable,rare_cat,frequent_cat):
    # create new variables, with freq labels imputed
    # by the most frequent category
    df[variable+'_freq_imp'] = np.where(df[variable].isin(rare_cat), frequent_cat, df[variable])

### 2. Missing Values Imputation

#### 2.1 Imputation for numerical attributes

In [25]:
# function for KNN model-based imputation of missing values using features without NaN as predictors
def impute_model_basic(df):
    cols_nan = df.columns[df.isna().any()].tolist()
    cols_no_nan = df.columns.difference(cols_nan).values
    for col in cols_nan:
        test_data = df[df[col].isna()]
        train_data = df.dropna()
        knr = KNeighborsRegressor(n_neighbors=5).fit(train_data[cols_no_nan], train_data[col])
        df.loc[df[col].isna(), col] = knr.predict(test_data[cols_no_nan])
    return df

In [26]:
# function for KNN model-based imputation of missing values using features without NaN as predictors,
#   including progressively added imputed features
def impute_model_progressive(df):
    cols_nan = df.columns[df.isna().any()].tolist()
    cols_no_nan = df.columns.difference(cols_nan).values
    while len(cols_nan) > 0:
        col = cols_nan[0]
        test_data = df[df[col].isna()]
        train_data = df.dropna()
        knr = KNeighborsRegressor(n_neighbors=5).fit(train_data[cols_no_nan], train_data[col])
        df.loc[df[col].isna(), col] = knr.predict(test_data[cols_no_nan])
        cols_nan = df.columns[df.isna().any()].tolist()
        cols_no_nan = df.columns.difference(cols_nan).values
    return df

In [27]:
# function for imputing missing data according to a given impute_strategy:
#  drop_rows: drop all rows with one or more missing values
#  drop_cols: drop columns with one or more missing values
#  model_basic: KNN-model-based imputation with fixed predictors
#  model_progressive: KNN-model-based imputation with progressively added predictors
#  mean, median, most_frequent: imputation with mean, median or most frequent values
#
#  cols_to_standardize: if provided, the specified columns are scaled between 0 and 1, after imputation
def impute_data(df_cleaned, impute_strategy=None, cols_to_standardize=None):
    df = df_cleaned.copy()
    if impute_strategy == 'drop_rows':
        df = df.dropna(axis=0)
    elif impute_strategy == 'drop_cols':
        df = df.dropna(axis=1)
    elif impute_strategy == 'model_basic':
        df = impute_model_basic(df)
    elif impute_strategy == 'model_progressive':
        df = impute_model_progressive(df)
    else:
        arr = SimpleImputer(missing_values=np.nan,strategy=impute_strategy).fit(
          df.values).transform(df.values)
        df = pd.DataFrame(data=arr, index=df.index.values, columns=df.columns.values)
    if cols_to_standardize != None:
        cols_to_standardize = list(set(cols_to_standardize) & set(df.columns.values))
        df[cols_to_standardize] = df[cols_to_standardize].astype('float')
        df[cols_to_standardize] = pd.DataFrame(data=MinMaxScaler().fit(
          df[cols_to_standardize]).transform(df[cols_to_standardize]),
                                             index=df[cols_to_standardize].index.values,
                                             columns=df[cols_to_standardize].columns.values)
    return df

In [28]:
df[numerical_attributes] = impute_data(df[numerical_attributes], 'model_progressive')

#### 2.2 Imputation for categorical attributes

In [29]:
def impute_na_freq(df, variable):
    # find out most frequent category
    most_frequent_category = df.groupby([variable])[variable].count().sort_values(ascending=False).index[0] 
    
    ## replace missing values with most frequent category
    df[variable].fillna(most_frequent_category, inplace=True)

In [30]:
def impute_na_addCat(df, variable):
    if((df[variable].isnull().sum())>0):
        df[variable+'_NA'] = np.where(df[variable].isnull(), 'Missing', df[variable])

In [31]:
for cols in categorical_attributes:
    impute_na_addCat(df,cols)
    impute_na_freq(df,cols)

### 3. Encoding of categorical attributes

In [32]:
def CategoricalEncoding_OneHot(df,variable):
    return pd.get_dummies(df, columns=[variable])

In [33]:
#Using weight of evidence encoding technique
def CategoricalEncoding_WOE(df,variable,target_variable):
    # now we calculate the probability of target=1 
    prob_df = df.groupby([variable])[target_variable].mean()
    prob_df = pd.DataFrame(prob_df)
    
    # and now the probability of target = 0 
    # and we add it to the dataframe
    prob_df['target_0'] = 1-prob_df[target_variable]
    prob_df.loc[prob_df[target_variable] == 0, target_variable] = 0.001
    prob_df['WoE'] = np.log(prob_df[target_variable]/prob_df['target_0'])
    ordered_labels = prob_df['WoE'].to_dict()
    df[variable+'_ordered'] = df[variable].map(ordered_labels)

In [34]:
#Replace labels by risk factor encoding technique
def CategoricalEncoding_RiskFactor(df,variable,target_variable):
    ordered_labels = df.groupby([variable])[target_variable].mean().to_dict()
    df[variable+'_ordered'] = df[variable].map(ordered_labels)

In [35]:
def CategoricalEncoding_Monotonicity(df,variable,target_variable):
    ordered_labels=df.groupby([variable])[target_variable].mean().sort_values().index
    ordinal_label = {k:i for i, k in enumerate(ordered_labels, 1)}
    df[variable+'_ordered']=df[variable].map(ordinal_label)

In [36]:
#Replace labels by risk factor encoding technique
def CategoricalEncoding_PRE(df,variable,target_variable):
    # now we calculate the probability of target=1 
    prob_df = df.groupby([variable])[target_variable].mean()
    prob_df = pd.DataFrame(prob_df)
    
    # and now the probability of target = 0 
    # and we add it to the dataframe
    prob_df['target_0'] = 1-prob_df[target_variable]
    prob_df.loc[prob_df['target_0'] == 0, 'target_0'] = 0.001
    prob_df['PRE'] = prob_df[target_variable]/prob_df['target_0']
    ordered_labels = prob_df['PRE'].to_dict()
    df[variable+'_ordered'] = df[variable].map(ordered_labels)

Sample dataset categorical attributes is already encoded, hence no need for this step.

In [37]:
# for col in ['Family', 'Education']:
#      df = CategoricalEncoding_OneHot(df,col)

### 4. Scaling of Attributes

In [38]:
# 1.Standard Scalar: z = (x - x_mean) / std
def scaler_Standard(df):
    # separate x and y
    df_x = df.drop(columns=[target])
    columns = df_x.columns
    index = df_x.index
    # the scaler - for standardisation
    from sklearn.preprocessing import StandardScaler
    # set up the scaler
    scaler = StandardScaler()
    # fit the scaler to the train set, it will learn the parameters
    scaler.fit(df_x)
    # transform train and test sets
    df_scaled = scaler.transform(df_x)
    # let's transform the returned NumPy arrays to dataframes 
    df_scaled = pd.DataFrame(df_scaled, columns=columns, index=index)
    # join back 
    df_scaled[target] = df[target]
    return df_scaled

In [39]:
# 2.Mean Normalisation: z=(x-x_mean)/(x_max-x_min)
def scaler_MeanNormalisation(df):
    # separate x and y
    df_x = df.drop(columns=[target])
    means = df_x.mean(axis=0)
    ranges = df_x.max(axis=0)-df_x.min(axis=0)
    df_scaled = (df_x - means) / ranges
    # join back 
    df_scaled[target] = df[target]
    return df_scaled

In [40]:
# 3.MinMaxScaling:x_scaled=(x-x_min)/(x_max-x_min)
def scaler_MinMax(df):
    # separate x and y
    df_x = df.drop(columns=[target])
    columns = df_x.columns
    index = df_x.index
    # the scaler - for min-max scaling
    from sklearn.preprocessing import MinMaxScaler
    # set up the scaler
    scaler = MinMaxScaler()
    # fit the scaler to the train set, it will learn the parameters
    scaler.fit(df_x)
    # transform train and test sets
    df_scaled = scaler.transform(df_x)
    # let's transform the returned NumPy arrays to dataframes 
    df_scaled = pd.DataFrame(df_scaled, columns=columns, index=index)
    # join back 
    df_scaled[target] = df[target]
    return df_scaled

In [41]:
# 4.MaxAbsScaling:x_scaled=x/x_max
def scaler_MaxAbs(df):
    # separate x and y
    df_x = df.drop(columns=[target])
    columns = df_x.columns
    index = df_x.index
    # the scaler - for min-max scaling
    from sklearn.preprocessing import MaxAbsScaler
    # set up the scaler
    scaler = MaxAbsScaler()
    # fit the scaler to the train set, it will learn the parameters
    scaler.fit(df_x)
    # transform train and test sets
    df_scaled = scaler.transform(df_x)
    # let's transform the returned NumPy arrays to dataframes 
    df_scaled = pd.DataFrame(df_scaled, columns=columns, index=index)
    # join back 
    df_scaled[target] = df[target]
    return df_scaled

In [42]:
# 5.RobustScaling:x_scaled = x - x_median / ( x.quantile(0.75) - x.quantile(0.25) )
def scaler_Robust(df):
    # separate x and y
    df_x = df.drop(columns=[target])
    columns = df_x.columns
    index = df_x.index
    # the scaler - for min-max scaling
    from sklearn.preprocessing import RobustScaler
    # set up the scaler
    scaler = RobustScaler()
    # fit the scaler to the train set, it will learn the parameters
    scaler.fit(df_x)
    # transform train and test sets
    df_scaled = scaler.transform(df_x)
    # let's transform the returned NumPy arrays to dataframes 
    df_scaled = pd.DataFrame(df_scaled, columns=columns, index=index)
    
    # join back 
    df_scaled[target] = df[target]
    return df_scaled

In [43]:
df_scaled = scaler_Robust(df)

### Save transformed dataset

In [44]:
df_scaled.head()

,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Personal Loan
ID,,,,,,,,,,,,,
1,-1.0,-0.95,-0.254237,-0.863923,1.0,0.055556,-0.5,0.0,1.0,0.0,-1.0,0.0,0
2,0.0,-0.05,-0.508475,-1.241379,0.5,0.000000,-0.5,0.0,1.0,0.0,-1.0,0.0,0
3,-0.3,-0.25,-0.898305,0.475714,-0.5,-0.277778,-0.5,0.0,0.0,0.0,-1.0,0.0,0
4,-0.5,-0.55,0.610169,0.250278,-0.5,0.666667,0.0,0.0,0.0,0.0,-1.0,0.0,0
5,-0.5,-0.60,-0.322034,-0.781238,1.0,-0.277778,0.0,0.0,0.0,0.0,-1.0,1.0,0


In [45]:
df_scaled.to_excel('../dataset/Bank_Personal_Loan_Modelling_transformed.xlsx')